In [ ]:
#use api for finding the best https://api.biorxiv.org

In [ ]:
#first find top ten papers 

Raw response text: <br />
<b>Warning</b>:  Undefined array key "d2" in <b>/var/www/vhosts/api.biorxiv.org/inc_published_details.php</b> on line <b>9</b><br />
{"messages":[{"status":"no articles found for 0 "}], "collection":[]}




Error decoding JSON: Expecting value: line 1 column 1 (char 0)
top_ten_papers []


In [ ]:
#get all DOIs from all bioarxiv papers from the current day

#https://www.biorxiv.org/content/10.1101/2024.09.17.613526v1
#https://doi.org/10.1101/2024.09.17.613526

In [ ]:
#create url for date of today
https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard

#get all urls

#iter over all urls and return the following data:
# "url": tweet["url"],
# "tweet_count": tweet["tweet_count"],
# "abstract": tweet["abstract"],
# "title": tweet["title"],
# "subject_area": tweet["subject_area"],

#

In [2]:
from datetime import datetime, timedelta
import pytz
import requests
from bs4 import BeautifulSoup
import asyncio
from playwright.async_api import (
    async_playwright,
    TimeoutError as PlaywrightTimeoutError,
)
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()


# Function to get date from yesterday ET time
def get_yesterday_date():
    eastern = pytz.timezone("US/Eastern")
    yesterday = datetime.now(eastern) - timedelta(days=1)
    return yesterday.strftime("%Y-%m-%d")


yesterday_date = get_yesterday_date()


# Construct the URL using yesterday_date
base_url = "https://www.biorxiv.org/search/jcode%3Abiorxiv"
limit_from_to = f"limit_from%3A{yesterday_date}%20limit_to%3A{yesterday_date}"
num_results = "numresults%3A75"
sort_order = "sort%3Arelevance-rank"
format_result = "format_result%3Astandard"

# Complete URL
complete_url = f"{base_url}%20{limit_from_to}%20{num_results}%20{sort_order}%20{format_result}"

print("Constructed URL:", complete_url)

Constructed URL: https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard


In [3]:
async def extract_pagination_urls():
    pagination_urls = []  # Initialize the variable to store pagination URLs
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=False)  # Change to True for headless mode
        context = await browser.new_context(viewport={"width": 1920, "height": 1080})  # Full-screen size
        page = await context.new_page()
        
        try:
            # Navigate to the complete URL
            await page.goto(complete_url)
            print(f"Successfully navigated to {complete_url}")

            # Wait for the page to fully load
            await page.wait_for_load_state('networkidle', timeout=20000)  # Wait for network to be idle

            # Get the entire HTML content of the page
            content = await page.content()
            soup = BeautifulSoup(content, 'html.parser')

            # Extract pagination links from the specified div
            pagination_div = soup.find('div', class_='highwire-list page-group-items item-list')
            if pagination_div:
                pagination_links = pagination_div.find('ul', class_='pager pager-items')
                
                # Add the current page URL
                pagination_urls.append(complete_url)

                for link in pagination_links.find_all('a'):
                    href = link.get('href')
                    if href:
                        # Add the base URL for pagination links that are not the current page
                        if not href.startswith('http'):
                            href = f"https://www.biorxiv.org{href}"
                        pagination_urls.append(href)

                print("Extracted Pagination URLs:", pagination_urls)  # Store the URLs in the local variable

        except PlaywrightTimeoutError:
            print("Navigation timed out.")
            return []  # Return an empty list in case of a timeout
        finally:
            await browser.close()

    return pagination_urls  # Return the pagination URLs

pagination_urls = await extract_pagination_urls()  # Call the function and store the result

Successfully navigated to https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard
Extracted Pagination URLs: ['https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard', 'https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard?page=1', 'https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard?page=2', 'https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_to%3A2024-09-19%20numresults%3A75%20sort%3Arelevance-rank%20format_result%3Astandard?page=3', 'https://www.biorxiv.org/search/jcode%3Abiorxiv%20limit_from%3A2024-09-19%20limit_t

In [6]:
from bs4 import BeautifulSoup  # Ensure BeautifulSoup is imported

async def open_pagination_urls():
    all_doi_urls = []  # List to store all extracted DOI URLs
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=False)  # Change to True for headless mode
        context = await browser.new_context(viewport={"width": 1920, "height": 1080})  # Full-screen size
        page = await context.new_page()

        for url in pagination_urls:
            try:
                await page.goto(url)

                # Wait for the page to fully load
                await page.wait_for_load_state('networkidle', timeout=20000)  # Wait for network to be idle

                # Get the entire HTML content of the page
                content = await page.content()
                soup = BeautifulSoup(content, 'html.parser')  # Parse the content with BeautifulSoup

                # Extract DOI URLs using BeautifulSoup
                doi_elements = soup.find_all('span', class_='highwire-cite-metadata-doi')
                for doi_element in doi_elements:
                    doi_link = doi_element.get_text(strip=True).replace("doi:", "").strip()  # Clean the text
                    if doi_link:  # Ensure the link is not empty
                        all_doi_urls.append(doi_link)  # Add the DOI link to the list

            except PlaywrightTimeoutError:
                print(f"Navigation to {url} timed out.")
            except Exception as e:
                print(f"An error occurred while navigating to {url}: {e}")

        await browser.close()
        print("all_doi_urls", all_doi_urls)
        print("len", len(all_doi_urls))
    
    return all_doi_urls  # Return the list of DOI URLs

all_doi_urls = await open_pagination_urls()  # Call the function to populate all_doi_urls

all_doi_urls ['https://doi.org/10.1101/2024.09.18.613606', 'https://doi.org/10.1101/2024.09.19.613919', 'https://doi.org/10.1101/2024.09.18.613694', 'https://doi.org/10.1101/2024.09.18.613705', 'https://doi.org/10.1101/2024.09.17.613573', 'https://doi.org/10.1101/2024.09.17.613540', 'https://doi.org/10.1101/2024.09.18.613641', 'https://doi.org/10.1101/2024.09.19.613976', 'https://doi.org/10.1101/2024.09.19.613956', 'https://doi.org/10.1101/2024.09.16.613038', 'https://doi.org/10.1101/2024.09.18.613759', 'https://doi.org/10.1101/2024.09.16.613364', 'https://doi.org/10.1101/2024.09.18.613805', 'https://doi.org/10.1101/2024.09.16.613363', 'https://doi.org/10.1101/2024.09.16.613180', 'https://doi.org/10.1101/2024.09.17.613505', 'https://doi.org/10.1101/2024.09.16.613336', 'https://doi.org/10.1101/2024.09.18.613741', 'https://doi.org/10.1101/2024.09.17.613313', 'https://doi.org/10.1101/2024.09.19.613857', 'https://doi.org/10.1101/2024.09.18.613782', 'https://doi.org/10.1101/2024.09.18.61361

In [11]:
def get_top_ten_tweets(tweet_data_list):
    # Sort the list in descending order based on tweet count
    sorted_tweets = sorted(
        tweet_data_list, key=lambda x: int(x["tweet_count"]), reverse=True
    )
    # Select the top ten URLs with the most tweets
    top_ten_tweets = sorted_tweets[:10]
    return top_ten_tweets

tweet_data_list = []

async def fetch_and_parse(url, context):
    print(f"Fetching URL: {url}")
    retries = 3
    for attempt in range(retries):
        try:
            page = await context.new_page()
            await page.goto(url, timeout=60000)  # Wait up to 60 seconds for the page to load
            
            # Wait for the tweet count element to be present
            await page.wait_for_selector("#count_twitter", timeout=60000)  # Wait for the element
            
            content = await page.content()
            await page.close()
            break  # Exit the retry loop if successful
        except PlaywrightTimeoutError as e:
            print(f"Timeout error on {url}: {e}")
            await page.screenshot(path=f"screenshot_{url.split('/')[-1]}.png")  # Save screenshot
            await page.close()  # Ensure the page is closed
        except Exception as e:
            print(f"Error on {url}: {e}")
            await page.screenshot(path=f"screenshot_{url.split('/')[-1]}.png")  # Save screenshot
            await page.close()  # Ensure the page is closed
        if attempt < retries - 1:
            print(f"Retrying {url} (attempt {attempt + 1}/{retries})")
        else:
            print(f"Failed to fetch {url} after {retries} attempts")
            return

    soup = BeautifulSoup(content, "html.parser")

    # Extract the date from the page
    date_element = soup.select_one(
        "#block-system-main > div > div > div > div > div:nth-child(2) > div > div > div:nth-child(3) > div"
    )
    if date_element:
        date_text = date_element.get_text(strip=True)
        # Remove the 'Posted\xa0' prefix if it exists
        if date_text.startswith("Posted\xa0"):
            date_text = date_text.replace("Posted\xa0", "")
        # Remove the period at the end of the date string
        if date_text.endswith("."):
            date_text = date_text[:-1]
        try:
            page_date = datetime.strptime(date_text, "%B %d, %Y").strftime("%Y-%m-%d")
            print(f"Parsed page date: {page_date}")  # Debug print
        except ValueError as e:
            print(f"Error parsing date on {url}: {e}")
            return

        # Check if the date matches the target date
        if page_date == yesterday_date:
            print(f"Date matches target date: {page_date}")  # Debug print

            # Extract the number of tweets using a CSS selector
            tweet_element = soup.select_one("#count_twitter")
            tweet_count = tweet_element.get_text(strip=True) if tweet_element else "0"
            print(f"Extracted tweet count: {tweet_count}")  # Debug print

            # Extract the abstract
            abstract_element = soup.select_one("#p-3")
            abstract = (
                abstract_element.get_text(strip=True) if abstract_element else "N/A"
            )
            print(f"Extracted abstract: {abstract}")  # Debug print

            # Extract the title
            title_element = soup.select_one("#page-title")
            title = title_element.get_text(strip=True) if title_element else "N/A"
            print(f"Extracted title: {title}")  # Debug print

            # Extract the subject area
            subject_area_elements = soup.select(
                "#block-system-main > div > div > div > div > div:nth-child(2) > div > div > div:nth-child(11) > div > div > div > ul > li > span > a"
            )
            subject_area = (
                ", ".join(
                    [element.get_text(strip=True) for element in subject_area_elements]
                )
                if subject_area_elements
                else "N/A"
            )
            print(f"Extracted subject area: {subject_area}")  # Debug print

            # Create a dictionary with the extracted data
            tweet_data = {
                "url": url,
                "tweet_count": tweet_count,
                "abstract": abstract,
                "title": title,
                "subject_area": subject_area,
            }

            # Add the dictionary to the list
            tweet_data_list.append(tweet_data)
        else:
            print(f"Date does not match target date on {url}")
    else:
        print(f"Date element not found on {url}")

# Main asynchronous function to handle multiple requests in batches
async def main(all_doi_urls, batch_size=50, delay=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

        for i in range(0, len(all_doi_urls), batch_size):
            batch = all_doi_urls[i : i + batch_size]
            print(f"Processing batch: {batch}")
            page = await context.new_page()
            tasks = [fetch_and_parse(url, context) for url in batch]
            await asyncio.gather(*tasks)
            await page.close()
            await asyncio.sleep(delay)

        await browser.close()

# Main function to run the entire process
async def get_trending_urls(all_doi_urls):  # Accept all_doi_urls as a parameter
    await main(all_doi_urls)  # Pass all_doi_urls to main
    return get_top_ten_tweets(tweet_data_list)

print(all_doi_urls)
top_ten_tweets = asyncio.run(get_trending_urls(all_doi_urls))  # Pass the list to the function
print("Top ten URLs with the most tweets:")
for data in top_ten_tweets:
    print(
        f"URL: {data['url']}, Tweets: {data['tweet_count']}, Title: {data['title']}, Abstract: {data['abstract']}, Subject Area: {data['subject_area']}"
    )

['https://doi.org/10.1101/2024.09.18.613606', 'https://doi.org/10.1101/2024.09.19.613919', 'https://doi.org/10.1101/2024.09.18.613694', 'https://doi.org/10.1101/2024.09.18.613705', 'https://doi.org/10.1101/2024.09.17.613573', 'https://doi.org/10.1101/2024.09.17.613540', 'https://doi.org/10.1101/2024.09.18.613641', 'https://doi.org/10.1101/2024.09.19.613976', 'https://doi.org/10.1101/2024.09.19.613956', 'https://doi.org/10.1101/2024.09.16.613038', 'https://doi.org/10.1101/2024.09.18.613759', 'https://doi.org/10.1101/2024.09.16.613364', 'https://doi.org/10.1101/2024.09.18.613805', 'https://doi.org/10.1101/2024.09.16.613363', 'https://doi.org/10.1101/2024.09.16.613180', 'https://doi.org/10.1101/2024.09.17.613505', 'https://doi.org/10.1101/2024.09.16.613336', 'https://doi.org/10.1101/2024.09.18.613741', 'https://doi.org/10.1101/2024.09.17.613313', 'https://doi.org/10.1101/2024.09.19.613857', 'https://doi.org/10.1101/2024.09.18.613782', 'https://doi.org/10.1101/2024.09.18.613618', 'https://

In [ ]:
#now i have all the code i need and i can put it together 